In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score


import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5779(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(Fore.YELLOW,Back.BLUE,'MakeSignal IB Version v1.55',Style.RESET_ALL)

print('')

try:
    LaBase = pd.read_csv('LaBase.csv')
    try:
        LaBase = LaBase.drop(['Unnamed: 0'],axis=1)
    except:
        pass
    try:
        LaBase = LaBase.drop(['Date.1'],axis=1)
    except:
        pass
except:
    print("Il n'y a pas de fichier LaBase")
    kill()

print('Sraping tickers')
compteur_csv = os.path.abspath('compteur.csv')
constituents = pd.read_csv('compteur.csv')
print('shape : ',constituents.shape)
constituents = constituents.drop_duplicates(['Symb.'])
try:
    constituents = constituents.drop(['Unnamed: 0'],axis=1)
except:
    pass
print('shape : ',constituents.shape)    
print('Scrap -----> ok')

today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
startday = str(dt.date.today()-dt.timedelta(1190))[:5]+str(dt.date.today()-dt.timedelta(1190))[5:7]+str(dt.date.today()-dt.timedelta(1190))[7:10]
yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
stamp = input("Enter the date (YYYY-MM-DD): ")

start = pd.to_datetime(startday)
end = pd.to_datetime(stamp)
print('Date et heure actuelle : ',time.strftime("%d/%m/%Y %H:%M:%S", time.localtime()))
print('La date scrapée est : ', stamp)


    
yaml.warnings({'YAMLLoadWarning': False})

def kill():
    os._exit

def scrap_data(ticker,start,end):
    
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)    
    return(df)

def boost(df):
    X = df
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = xgb.XGBRegressor(n_estimators=20000, objective='reg:squarederror',learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100)

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)

def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        #try:
        rsi70_list.append(70 - rsi.iloc[i])
        rsi30_list.append(rsi.iloc[i] - 30)
        #except:
         #   rsi70_list.append(0)
          #  rs30_list.append(0)
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    df = boost(df)
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]-5):
        if df.iloc[i]['%Futur'] > 0.1 :#or df.iloc[i]['%Futur2'] > 0.1:
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.1: #or df.iloc[i]['%Futur2'] < -0.1:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    
    
    df['target_up'] = up  # target_up # abs(np.array(valley))#target_up
    df['target_down'] = down # target_down # abs(np.array(peak))#target_down
    #df = df.dropna()
    return(df)

def deep_learning(df):
    X = df.copy()
    X = X.iloc[:,1:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    seed = 770
    np.random.seed(seed)

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    #accuracy_up = accuracy_score(ytest_up, yhat_up)
    #accuracy_down = accuracy_score(ytest_down, yhat_down)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    #recall_up = recall_score(ytest_up, yhat_up) 
    #recall_down = recall_score(ytest_down, yhat_down) 


    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

tmps55=time.time()

signals = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short'])
global delta,bloc1,bloc2,model_up,model_down

GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
i = -1

warnings.filterwarnings("ignore")

print('The computing has begun at ',dt.datetime.now().hour,':',dt.datetime.now().minute,':',dt.datetime.now().second)


    
print('Le denier signal est ',LaBase.iloc[-1]['Date'])

for loop in range(0, len(constituents)):
#for loop in range(0, 2):
    
    if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == 50:
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == 100:
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == 150:
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == 200:
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == 250:
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == 300:
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == 350:
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == 400:
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',end='',flush=True)
    
    
    #try:
    ticker = (constituents.iloc[loop]['Symb.'])
    #name = constituents.iloc[loop]['Name']
    #sector = constituents.iloc[loop]['Sector']

    yamlup = os.path.join(os.path.dirname(compteur_csv), 'DONNEES_DL/Save_'+ticker+'_up.yaml')
    yamldown = os.path.join(os.path.dirname(compteur_csv), 'DONNEES_DL/Save_'+ticker+'_down.yaml')
    modelup = os.path.join(os.path.dirname(compteur_csv), 'DONNEES_DL/Save_'+ticker+'_up.h5')
    modeldown = os.path.join(os.path.dirname(compteur_csv), 'DONNEES_DL/Save_'+ticker+'_down.h5')

    yaml_file_up = open(yamlup, 'r')
    yaml_file_down = open(yamldown, 'r')
    model_yaml_up = yaml_file_up.read()
    model_yaml_down = yaml_file_down.read()
    yaml_file_up.close()
    yaml_file_down.close()
    model_up = model_from_yaml(model_yaml_up)
    model_down = model_from_yaml(model_yaml_down)
    # load weights into new model
    model_up.load_weights(modelup)
    model_down.load_weights(modeldown)


    #df = scrap_data(ticker,start,end)
    
    df = LaBase[LaBase['Symbol']==ticker]
    df = df.drop(['Symbol'],axis=1)
    df = df.iloc[-820:,:]
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)
    df = df.set_index('Date')

    #if str(df.index[-1])[:10] != today:
     #   print(Fore.RED,'PROBLEME DE DATE')
      #  break
       # kill()

    delta = round(df.shape[0])
    bloc1 = round(delta*0.80)
    bloc2 = delta - bloc1

    df = prepa_data(df)

    resultats,precision_up,precision_down,scaler = deep_learning(df)
    
    price = df.iloc[i]['Close']

    if (precision_up * 100) > 73 and (precision_down * 100) > 73:

        ####################
        ##### SIGNALS #####
        ###################

        filtre_up = 93
        filtre_down = 97
        filtre_up_close = 80
        filtre_down_close = 80


        if resultats.iloc[i]['Confiance Down'] > filtre_down:
            doob = -1
        elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
            waab = -1
            doob = 0
        elif resultats.iloc[i]['Confiance up'] > filtre_up:
            doob = 1
        elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
            waab = 1
            doob = 0
        else :
            doob = 0
            waab = 0

        if doob == 1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(1)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(1)
            PRICE.append(price)

        elif doob == -1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(1)
            GREEN_CLOSE.append(1)
            RED_CLOSE.append(0)
            PRICE.append(price)

        elif doob == 0 and waab == -1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(1)
            PRICE.append(price)

        elif doob == 0 and waab == 1 :
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(1)
            RED_CLOSE.append(0)
            PRICE.append(price)

        else:
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(0)
            PRICE.append(price)


    else:
        DATE.append(df.index[i])
        TICKER.append(ticker)
        GREEN.append(0)
        RED.append(0)
        GREEN_CLOSE.append(0)
        RED_CLOSE.append(0)
        PRICE.append(price)
            

    #except:
     #   continue


signals['Date'] = DATE
signals['Ticker'] = TICKER
signals['Open Long'] = GREEN
signals['Close Long'] = GREEN_CLOSE
signals['Open Short'] = RED
signals['Close Short'] = RED_CLOSE 
signals['Close Price'] = PRICE

try:
    
    signals = pd.read_csv('SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    print('Le fichier SIGNALS_'+stamp+'.csv existe déjà')
    
    
except:
    print('\n Création du fichier SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    signals.to_csv('SIGNALS_'+stamp+'.csv')        

tmps22=round(time.time()-tmps55,2)
print("Time for complete Signals Generation = %f" %tmps22,'seconds \n')

Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5779(c)
 °0Oo_D.A.G._26_oO0°
  MakeSignal IB Version v1.55 

Sraping tickers
shape :  (500, 6)
shape :  (373, 5)
Scrap -----> ok


Enter the date (YYYY-MM-DD):  2019-12-23


Date et heure actuelle :  27/12/2019 11:51:34
La date scrapée est :  2019-12-23
The computing has begun at  11 : 51 : 34
Le denier signal est  2019-12-23
   |================  O  ====|Le fichier SIGNALS_2019-12-23.csv existe déjà
Time for complete Signals Generation = 822.150000 seconds 



In [9]:
signals[(signals['Close Long']==1)|(signals['Close Short']==1)]

,Unnamed: 0,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Close Price
12,12,2019-12-23,ADS,0,1,1,0,111.470001
19,19,2019-12-23,AIG,1,0,0,1,50.970001
29,29,2019-12-23,AMD,0,1,1,0,45.459999
30,30,2019-12-23,AME,0,1,1,0,100.139999
34,34,2019-12-23,AMZN,1,0,0,1,1793.000000
40,40,2019-12-23,APTV,0,1,1,0,95.529999
61,61,2019-12-23,C,0,1,1,0,78.760002
62,62,2019-12-23,CAH,0,1,1,0,51.130001
66,66,2019-12-04,CBS,0,1,1,0,40.770000
75,75,2019-12-23,CHRW,0,0,0,1,76.809998


In [8]:
sum(GREEN),sum(GREEN_CLOSE),sum(RED),sum(RED_CLOSE)

(11, 20, 19, 13)

In [ ]:
resultats

In [ ]:
len(signals[1==signals['Open Long']])

In [ ]:
str(web.DataReader('ZION','yahoo',yesterday,today).index[-1])[:10]

In [ ]:
df = LaBase[LaBase['Symbol']==ticker]
df.tail()

In [ ]:
df.tail()

In [ ]:
today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
startday = str(dt.date.today()-dt.timedelta(1190))[:5]+str(dt.date.today()-dt.timedelta(1190))[5:7]+str(dt.date.today()-dt.timedelta(1190))[7:10]
yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
stamp = str(web.DataReader('ZION','yahoo',yesterday,today).index[-1])[:10]

today, startday, yesterday, stamp

In [ ]:
loop

In [ ]:
rsi = talib.RSI(df['Close'],timeperiod=14)
rsi.iloc[60]
                

In [ ]:
X.head()

In [ ]:
X = df.copy()
X = X.iloc[:,:-4]
X.head()

In [ ]:
X.astype(np.float64)
y_up = df.iloc[:,-2].values
y_down = df.iloc[:,-1].values


In [ ]:
dfori = pd.DataFrame()
dfori = web.DataReader(ticker,'yahoo',start,end)
dfori = dfori.drop(['Close'],axis=1)
dfori['Close'] = dfori['Adj Close']
dfori = dfori.drop(['Adj Close'],axis = 1)
dfori.head()

In [ ]:
LaBase = pd.read_csv('LaBase.csv')
ticker = 'A'
df = LaBase[LaBase['Symbol']==ticker]
df = df.drop(['Symbol'],axis=1)
df = df.iloc[-820:,:]
df = df.drop(['Close'],axis=1)
df['Close'] = df['Adj Close']
df = df.drop(['Adj Close'],axis = 1)
df = df.set_index('Date')
df.head()

In [ ]:
df.shape

In [ ]:
X.shape

In [ ]:
end

In [ ]:
ticker = (constituents.iloc[28]['Symb.'])
df = LaBase[LaBase['Symbol']==ticker]
len(df),ticker

In [ ]:
df.tail()

In [ ]:
stamp,end

In [4]:
LaBase.shape

(408471, 8)

In [6]:
LaBase = LaBase.drop_duplicates()

In [7]:
LaBase.shape

(408471, 8)